## ここはいろいろ実験的なコードを書くところとする。

In [1]:
import numpy as np
import pandas as pd
train_genba = pd.read_csv("data/train_genba.tsv",sep='\t')

In [2]:
df = train_genba.copy()

In [7]:
df = df.drop('jukyo',axis=1)

In [39]:
# df['yoto'], df['kempei'],df['yoseki']= df.apply(select_yoto,axis=1)
df[['yoto','kempei','yoseki']] = df.apply(select_yoto,axis=1)

In [40]:
df.head()

,pj_no,chiseki_js_hb,chiseki_kb_hb,yoto1,yoto2,kempei1,kempei2,yoseki1,yoseki2,josui,...,teiho1,rosen_nm2,eki_nm2,bas_toho2,eki_kyori2,bastei_nm2,teiho2,yoto,kempei,yoseki
0,0,109.26,109.10,工業地域,NaN,60,0.0,200,0.0,公営,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,工業地域,60,200
1,1,136.15,136.00,第一種低層住居専用地域,NaN,60,0.0,200,0.0,公営,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,第一種低層住居専用地域,60,200
2,2,86.92,86.52,第一種低層住居専用地域,NaN,50,0.0,100,0.0,私営,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,第一種低層住居専用地域,50,100
3,3,164.91,164.07,工業地域,NaN,60,0.0,200,0.0,公営,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,工業地域,60,200
4,4,403.58,403.58,第一種低層住居専用地域,NaN,50,0.0,80,0.0,公営,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,第一種低層住居専用地域,50,80


In [41]:
org_df = df.copy()

In [43]:
df['josui'] = df.apply(lambda x: 1 if x['josui']=='公営' else 0, axis=1)

In [46]:
df['gesui'] = df.apply(lambda x: 1 if x['gesui']=='公共下水' else 0, axis=1)

In [47]:
df['gas'] = df.apply(lambda x: 2 if x['gas'] =='都市ガス' else 1 if x['gas'] == '集中プロパン' else 0, axis=1)

In [48]:
df['chiseki_diff'] = df['chiseki_kb_hb']-df['chiseki_js_hb']

In [63]:
a = pd.get_dummies(df[['usui']],drop_first=False)

In [65]:
df2 = pd.merge(df, a, left_index=True, right_index=True)

In [68]:
df = pd.merge(df, pd.get_dummies(df[['usui']],drop_first = False), left_index = True, right_index = True)

In [75]:
df = df.fillna({'tateuri_su': 0})

In [82]:
df = pd.merge(df, pd.get_dummies(df[['road1_hk','road1_sb']],drop_first = False), left_index = True, right_index = True)

TypeError: merge() got an unexpected keyword argument 'axis'

In [81]:
df['road_num']=df.apply(lambda x: 4 if x['road4_hk']!=np.nan else 3)

KeyError: ('road4_hk', 'occurred at index pj_no')

In [98]:
df['road_num']=df.apply(lambda x: 4 if x['road4_fi']!=0 else 3 if x['road3_fi']!=0 else 2 if x['road2_fi']!=0 else 1, axis=1)

In [94]:
import math

In [96]:
df['road_num']=df.apply(lambda x: 4 if not math.isnan(x['road4_fi']) else 3 if not math.isnan(x['road3_fi']) else 2 if not math.isnan(x['road2_fi']) else 1, axis=1)

In [143]:
df['toshikuiki']=df['toshikuiki2'].fillna('empty')

In [145]:
df['toshikuiki'] = df.apply(lambda x: x['toshikuiki1'] if x['toshikuiki']=="empty" else x['toshikuiki2'], axis=1)

In [146]:
df['toshikuiki']

0         市街化区域
1         市街化区域
2         市街化区域
3         市街化区域
4         市街化区域
5         市街化区域
6       市街化調整区域
7         市街化区域
8         市街化区域
9       市街化調整区域
10        市街化区域
11        市街化区域
12        市街化区域
13        市街化区域
14        市街化区域
15        市街化区域
16      市街化調整区域
17        市街化区域
18        市街化区域
19        市街化区域
20        市街化区域
21        市街化区域
22        市街化区域
23        市街化区域
24        市街化区域
25        市街化区域
26        市街化区域
27        市街化区域
28      市街化調整区域
29        市街化区域
         ...   
2751      市街化区域
2752      市街化区域
2753      市街化区域
2754      市街化区域
2755      市街化区域
2756      市街化区域
2757      市街化区域
2758      市街化区域
2759      市街化区域
2760      市街化区域
2761      市街化区域
2762      市街化区域
2763      市街化区域
2764     非線引き区域
2765      市街化区域
2766      市街化区域
2767      市街化区域
2768      市街化区域
2769      市街化区域
2770      市街化区域
2771      市街化区域
2772    市街化調整区域
2773      市街化区域
2774      市街化区域
2775      市街化区域
2776      市街化区域
2777      市街化区域
2778      市街化区域
2779      市街化区域
2780      市街化区域
Name: toshikuiki, Length

In [119]:
df.loc[1549,'toshikuiki2']

'準都市計画区域'

In [38]:
def select_yoto(x):
    if ( x['yoto2']==np.nan):
        yoto =x['yoto1']
        kenpei =x['kempek1']
        yoseki =x['yoseki1']
    elif ( x['kempei1']<x['kempei2']):
        yoto =x['yoto1']
        kempei = x['kempei1']
        yoseki =x['yoseki1']
    else:
        yoto = x['yoto']=x['yoto1']
        kempei = x['kempei']=x['kempei1']
        yoseki =x['yoseki1']
    return pd.Series([yoto, kempei, yoseki])
